## Classifying Text: Supervised Classification

For this lesson, we will focus on how to build our first automatic classification algorithms. Since the topic is huge, we will be simply scratching the surface, to get something working. For those interested in learning more, taking the Data Mining course next semester is the natural sequence.

Classification is the task of choosing the correct class label for a given input. In basic classification tasks, each input is considered in isolation from all other inputs, and the set of labels is defined in advance. Some examples of classification tasks are:

* Deciding whether an email is spam or not.
* Deciding what the topic of a news article is, from a fixed list of topic areas such as "sports," "technology," and "politics."
* Deciding whether a given occurrence of the word bank is used to refer to a river bank, a financial institution, the act of tilting to the side, or the act of depositing something in a financial institution.

A classifier is called supervised if it is built based on **training data** containing the correct label for each input. 

<img src="http://www.nltk.org/images/supervised-classification.png" width="50%">

### Document Classification

A common classification task is to classify documents into categories. Let's use for this the Movie Reviews corpus from NLTK:

In [1]:
from nltk.corpus import movie_reviews

categories = movie_reviews.categories()
categories

['neg', 'pos']

Now let's generate the list of files, each with its corresponding category.

In [2]:
labeled_files = []
for category in categories:
    labeled_files += [(fileid, category) for fileid in movie_reviews.fileids(category)]

In [3]:
labeled_files[:5]

[('neg/cv000_29416.txt', 'neg'),
 ('neg/cv001_19502.txt', 'neg'),
 ('neg/cv002_17424.txt', 'neg'),
 ('neg/cv003_12683.txt', 'neg'),
 ('neg/cv004_12641.txt', 'neg')]

In [4]:
labeled_files[-5:]

[('pos/cv995_21821.txt', 'pos'),
 ('pos/cv996_11592.txt', 'pos'),
 ('pos/cv997_5046.txt', 'pos'),
 ('pos/cv998_14111.txt', 'pos'),
 ('pos/cv999_13106.txt', 'pos')]

In [5]:
len(labeled_files)

2000

In [6]:
len([l for l in labeled_files if l[1]=='pos'])

1000

In [7]:
len([l for l in labeled_files if l[1]=='neg'])

1000

### Featurizing a Document

Now let's create the features. We will create **one feature per word**, with a **binary value**, indicating whether the document contains the word or not. To limit the number of features that the classifier needs to process, we begin by constructing a list of the 2000 most frequent words in the overall corpus

In [8]:
import nltk
from nltk.corpus import stopwords
import random

In [9]:
# Extract all the words from all the reviews
all_words = []
for (fileid, category) in labeled_files:
    # Get the words of the document
    all_words.extend(movie_reviews.words(fileid))

# Keep only words that are not stopwords
stopwords = nltk.corpus.stopwords.words('english')
filtered = [w.lower() for w in all_words if w.isalpha() and w not in stopwords]
    
# Compute the word frequency after removing stopwords 
fdist  = nltk.FreqDist(filtered)
features = set([w for (w,f) in fdist.most_common(2000)])

In [10]:
def document_features(fileid):
    # Get the words of the document
    document_words = set(movie_reviews.words(fileid))
    document_features = {}
    for word in features:
        # Create a boolean feature that is True when the document contains the word
        if word in document_words:
            document_features[word] = True
        else:
            document_features[word] = False
    return document_features

Let's see how long it takes to featurize a single document

In [11]:
%timeit  document_features("pos/cv995_21821.txt")

1.28 ms ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


And to visualize how a "featurized" document looks like:

In [12]:
import pandas as pd
testdf = pd.DataFrame(
    [document_features("pos/cv995_21821.txt"),
     document_features("neg/cv003_12683.txt")]
)
testdf

,ability,able,absolutely,academy,accent,accept,accident,across,act,acted,...,x,yeah,year,years,yes,yet,york,young,younger,zero
0,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [13]:
labeled_documents = [(document_features(fileid), category) for (fileid, category) in labeled_files]
random.shuffle(labeled_documents)
train_set, test_set = labeled_documents[100:], labeled_documents[:100]

In [14]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [15]:
print(nltk.classify.accuracy(classifier, test_set))

0.79


In [16]:
classifier.show_most_informative_features(50)

Most Informative Features
             outstanding = True              pos : neg    =     10.1 : 1.0
                  seagal = True              neg : pos    =      7.4 : 1.0
                   mulan = True              pos : neg    =      7.0 : 1.0
             wonderfully = True              pos : neg    =      6.8 : 1.0
                   damon = True              pos : neg    =      6.3 : 1.0
                  wasted = True              neg : pos    =      5.4 : 1.0
                    jedi = True              pos : neg    =      5.3 : 1.0
              ridiculous = True              neg : pos    =      5.2 : 1.0
                  poorly = True              neg : pos    =      5.1 : 1.0
                   awful = True              neg : pos    =      5.1 : 1.0
                    lame = True              neg : pos    =      4.9 : 1.0
                   waste = True              neg : pos    =      4.8 : 1.0
                   bland = True              neg : pos    =      4.4 : 1.0

### Exercise

Try to come up with ways to improve the classifier that we discussed above. 

In [17]:
# Extract all the words from all the reviews
all_words = []
for (fileid, category) in labeled_files:
    # Get the words of the document
    all_words.extend(movie_reviews.words(fileid))

# Keep only words that are not stopwords
stopwords = nltk.corpus.stopwords.words('english')
# Extend with a few keywords that are named entities
stopwords.extend(["mulan", "seagal", "damon", "ripley", "jedi", "hanks"])
filtered = [w.lower() for w in all_words if w.isalpha() and w not in stopwords]
    
# Compute the word frequency after removing stopwords 
fdist  = nltk.FreqDist(filtered)
features = set([w for (w,f) in fdist.most_common(2000)])

In [18]:
def document_features(fileid):
    # Get the words of the document
    document_words = set(movie_reviews.words(fileid))
    document_features = {}
    for word in features:
        # Create a boolean feature that is True when the document contains the word
        if word in document_words:
            document_features[word] = True
        else:
            document_features[word] = False
    return document_features

In [19]:
labeled_documents = [(document_features(fileid), category) for (fileid, category) in labeled_files]
random.shuffle(labeled_documents)
train_set, test_set = labeled_documents[100:], labeled_documents[:100]

classifier = nltk.NaiveBayesClassifier.train(train_set)
print("Accuracy:", nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(50)

Accuracy: 0.81
Most Informative Features
             outstanding = True              pos : neg    =     10.8 : 1.0
             wonderfully = True              pos : neg    =      7.5 : 1.0
                   flynt = True              pos : neg    =      5.6 : 1.0
                   awful = True              neg : pos    =      5.3 : 1.0
                  wasted = True              neg : pos    =      5.2 : 1.0
                    lame = True              neg : pos    =      5.1 : 1.0
                  poorly = True              neg : pos    =      4.8 : 1.0
              ridiculous = True              neg : pos    =      4.7 : 1.0
                   waste = True              neg : pos    =      4.7 : 1.0
                terrific = True              pos : neg    =      4.5 : 1.0
               portrayal = True              pos : neg    =      4.5 : 1.0
                     era = True              pos : neg    =      4.4 : 1.0
                   bland = True              neg : pos    =